# Predicting Cancer with DNA Methylation

This notebook is a walk-through of our project submitted to the 2020 Hack for Social Good
 _Build Hackathon.

This notebook is organized the following way:
 1. Data preparation
 2. Preprocessing
 3. Model Training and Evaluation
 4. Model Deployment

## Environment setup
This notebook has been tested with Python 3.8.5. If not done already, create the `DNA_Methylation`
virtual environment using conda and the `environment.yml` file.

In [6]:
import pandas as pd
import os

from google.cloud import bigquery, storage
import google.auth

from configs import configs

We tested this notebook with:
 - Pandas version used: 1.0.5

In [2]:
print(f"Pandas version used: {pd.__version__}")

Pandas version used: 1.1.2


Retrieve credentials to use GCP's Python libraries

In [3]:
credentials, project_id = google.auth.default()
print(project_id)

gcp-nyc


In [4]:
PROJECT_ID = configs.PROJECT_ID
DATASET = configs.DATASET

## Data preparation

In the data preparation part, we will create two bigquery tables that will used to create the
final train and test datasets.

The first BigQuery table is called `tcga_betas` is a BigQuery partitioned and clustered table
that will hold all betas values for all TCGA patients. It will be a table in a long format:
one row per betas observation.

The second BigQuery table is called `columns_to_keep` is a BigQuery table that will hold the
list of CpG site that we will keep in the final dataset.

### First table - `tcga_betas` table: a simple SQL script

This first table will be created using a simple SQL query. The query is located in
`SQL_queries/tcga_betas.txt`.

In [8]:
SQL_query_path = 'sql_queries/tcga_betas.txt'
with open(SQL_query_path, 'r') as f:
    sql_query = ' '.join(f.readlines())

sql_query = sql_query.replace('__DATASET__', DATASET)
sql_query = sql_query.replace('__TABLE_NAME__', 'tcga_betas')
print(sql_query)

CREATE TABLE dna_cancer_prediction.tcga_betas
 PARTITION BY RANGE_BUCKET(row_number, GENERATE_ARRAY(1, 11000, 100))
 CLUSTER BY row_number
 AS
 
 WITH
   brca_betas AS (
   SELECT
     MIM.CpG_probe_id,
     dna.case_barcode,
     dna.sample_barcode,
     dna.probe_id,
     dna.beta_value,
     gc.Name
   FROM
     `isb-cgc.platform_reference.methylation_annotation` gc
   JOIN
     `isb-cgc.TCGA_hg38_data_v0.DNA_Methylation` dna
   ON
     gc.Name = dna.probe_id
   JOIN
     `isb-cgc.platform_reference.GDC_hg38_methylation_annotation` MIM
   ON
     MIM.CpG_probe_id = gc.Name),
 
   participants_id AS (
   SELECT
     DISTINCT case_barcode
   FROM
     `isb-cgc.TCGA_hg38_data_v0.DNA_Methylation`),
 
   participants_id_numbered AS (
   SELECT
     case_barcode,
     ROW_NUMBER() OVER (order by case_barcode) as row_number
   FROM
     participants_id)
 
 SELECT
   A.case_barcode,
   A.beta_value,
   A.CpG_probe_id,
   A.sample_barcode,
   SUBSTR(A.sample_barcode, 14, 2) AS sample_id,
   

TO DO: describe query

We can execute the job to create the table in BigQuery.

In [13]:
client = bigquery.Client()
query_job = client.query(sql_query)


### Second table - `columns_to_keep`: using dataproc

The second table will be created by a Dataflow job. This table will hold the list of CpG sites
that will be included in the training dataset.

Indeed, we want to keep only the 5,000 best CpG site among the 500,000 that will be able to
separate cancerous vs non-cancerous observations.

In [ ]:
!gcloud dataproc jobs submit pyspark --cluster build-hackathon-nyc-cluster \
    --region us-east1 --jars gs://spark-lib/bigquery/spark-bigquery-latest.jar \
    dataproc_processing/create_cpg_sites_list.py \
    -- -gcs_bucket "build_hackathon_dnanyc_tfx_pipeline"

### Creating training dataset

We will now use those training tables to create our training dataset. The training dataset will be
saved into a GCS bucket.

In [9]:
DATASET_PATH = configs.DATASET_PATH
DATASET_NAME = 'tcga-binary.csv'

In [10]:
def configure_gcs(project_id=PROJECT_ID):
    client = storage.Client(project=project_id)
    return client

def save_to_gcs(df, gcs_path, file_name):
    df.to_csv(gcs_path + file_name)

In [38]:
def download_from_bigquery(project_id, list_of_columns):
    formatted_columns = "', '".join(list_of_columns)
    query = f"""
    SELECT beta_value, CpG_probe_id, sample_barcode
    from `dna_cancer_prediction.tcga_betas`
    where CpG_probe_id in ('{formatted_columns}')
    """
    df = pd.read_gbq(query, project_id=project_id)
    return df

def download_columns_to_keep(project_id):
    query = """
    SELECT CpG_probe_id
    from `dna_cancer_prediction.cpg_site_list`
    """
    df = pd.read_gbq(query, project_id=project_id)
    return df['CpG_probe_id'].values

def merge_and_pivot(df_betas):
    # TO DO: use aliquot_barcode instead of sample barcode as identifier
    df_betas = df_betas.drop_duplicates(subset=['sample_barcode', 'CpG_probe_id'])
    df_p = df_betas.pivot(index="sample_barcode", columns='CpG_probe_id', values='beta_value').reset_index()
    df_p['case_barcode'] = df_p['sample_barcode'].str[:12]
    df_final = df_p.merge(df_betas[['case_barcode', 'sample_status']],
                          how='left', on='case_barcode')
    df_final = df_final.drop('case_barcode', axis=1)
    df_final = df_final.set_index('sample_barcode')
    return df_final

In [24]:
gcs_client = configure_gcs(PROJECT_ID)
columns = download_columns_to_keep(PROJECT_ID)
df = download_from_bigquery(PROJECT_ID, columns)

In [ ]:
df = merge_and_pivot(df)
save_to_gcs(df, DATASET_PATH, DATASET_NAME)